# Modelo Preditivo de Fraude - Marcador de POC (DRAFT)
Random Forest sem Cross validation

## Objetivo
Os POCs reportados são focados em agências com maior ocorrência de fraude, não constituindo, portanto, um registro completo dos POCs ocorridos. Esta análise tem por objetivo  a melhorar a performance do modelo preditivo, a partir da marcação dos POCs de forma mais completa, por meio da análise dos eventos ocorridos nas agências/ATMs com POCs reportados.

## Libraries

In [1]:
import pyspark
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import HiveContext
from pyspark.sql.types import Row
from pyspark.sql.types import *
from pyspark.sql.types import DoubleType
from pyspark.sql.types import DateType
from pyspark.sql.types import IntegerType
from pyspark.sql.types import TimestampType
from pyspark.sql.functions import *
#import matplotlib as mpl
#import matplotlib.pyplot as plt
import atexit
from numpy import array
import numpy as np
import pandas as pd
from pyspark.mllib.linalg import Vectors
from pylab import plot,show,hist,figure,title
import unicodedata
from datetime import datetime, time
from pyspark.sql.functions import col,udf, unix_timestamp
from pyspark.ml import Pipeline
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline
from pyspark.sql import HiveContext 
hive_context = HiveContext(sc)
# para plots
%matplotlib inline

## Carga de Dados Brutos Hadoop Produção

In [2]:
# PARA HADOOP PRODUCAO USAR ABAIXO
funcStrToDate_mdyy =  udf (lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S.%f')\
                           .replace(minute=0, hour=0, second=0, microsecond=0), DateType())
funcStrToDate_dmyyyy =  udf (lambda x: datetime.strptime(x, '%m/%d/%y'), DateType())

In [3]:
# PARA HADOOP PRODUCAO USAR ABAIXO
rdd_eventlog = hive_context.table('default.ext_eventos_fullog1908')
poc_in = '/user/x181472/export/BASE_POC_ATM_090816.csv'
tranlog_in = hive_context.table('default.ext_juntatudo12')
tabelao_out = '/temporario/tabelao-temp-V5.2.7.csv'
previsao = '/temporario/previsao-V5.2.7.csv'
#tabelao_in = '/user/x181472/export/tabelao-temp-V5.2.7.csv'
tabelao_in = '/user/x184166/tabelao-temp-V5.2.7.csv' # Hermann

In [4]:
# PARA HADOOP PRODUCAO USAR ABAIXO
df_eventlog = rdd_eventlog \
    .map(lambda p: (p[1],p[2],p[3], p[4])) \
    .toDF(['nr_sequ_even',
           'dh_even', 
           'cd_eqpt', 
           'nr_idef_secu' 
                   ])
df_eventlog.printSchema() 

print ("\n\nNumero de linhas carregadas: " + str("%d" % (rdd_eventlog.count())))


root
 |-- nr_sequ_even: long (nullable = true)
 |-- dh_even: string (nullable = true)
 |-- cd_eqpt: string (nullable = true)
 |-- nr_idef_secu: string (nullable = true)



Numero de linhas carregadas: 18341921


In [5]:
# PARA HADOOP PRODUCAO USAR ABAIXO
# sumarizar a base por agencia+atm+evento+dia
df_eventlog = df_eventlog\
    .groupBy(['nr_idef_secu', 'cd_eqpt', 'nr_sequ_even', 'dh_even']).count()\
    .orderBy(col('nr_idef_secu'), col('cd_eqpt'), col('nr_sequ_even'), col('dh_even').desc())\
    .withColumnRenamed('count', 'nr_eventos')


## Carga de dados Brutos MAC

In [6]:
# PARA MAC USAR ABAIXO
#eventlog_in = '/Users/josecassiodossantosjunior/Documents/spark-2.0.0-bin-hadoop2.7/MySpark/dados/tbfullog.csv'
#poc_in = '/Users/josecassiodossantosjunior/Documents/spark-2.0.0-bin-hadoop2.7/MySpark/dados/BASE_POC_ATM_090816.csv'
#tranlog_in = '/Users/josecassiodossantosjunior/Documents/spark-2.0.0-bin-hadoop2.7/dados/juntatudo11pordia-tranfin.csv'
#tabelao_out = '/Users/josecassiodossantosjunior/Documents/spark-2.0.0-bin-hadoop2.7/MySpark/dados/tabelao_stdr-V5.2.2.csv'
#tabelao_in = '/Users/josecassiodossantosjunior/Documents/spark-2.0.0-bin-hadoop2.7/MySpark/dados/tabelao_stdr-V5.2.2.csv'
#previsao = '/Users/josecassiodossantosjunior/Documents/spark-2.0.0-bin-hadoop2.7/MySpark/dados/previsao-V5.2.2.csv'

In [7]:
# PARA MAC
#funcStrToDate_mdyy =  udf (lambda x: datetime.strptime(x, '%d/%m/%Y'), DateType())
#funcStrToDate_dmyyyy =  udf (lambda x: datetime.strptime(x, '%m/%d/%y'), DateType())

In [8]:
# PARA MAC USAR ABAIXO
#rdd_eventlog = sc.textFile(eventlog_in)
#print ("\n\nNúmero de linhas carregadas: " + str("%d" % (rdd_eventlog.count())))

#header = rdd_eventlog.first()

#df_eventlog = rdd_eventlog \
#    .filter(lambda line: line != header) \
#    .map(lambda k: k.split(",")) \
#    .map(lambda p: (p[1],p[2],p[3],p[4], p[5])) \
#    .toDF(['nr_sequ_even',
#            'dh_even', 
#            'cd_eqpt', 
#            'nr_idef_secu', 
#            'nr_eventos'
#                   ])
#df_eventlog.printSchema()    

In [9]:
df_eventlog = df_eventlog.withColumn('dh_even', funcStrToDate_mdyy(col('dh_even'))) 

In [10]:
df_eventlog = df_eventlog \
    .withColumn('nr_sequ_even', df_eventlog["nr_sequ_even"].cast(StringType()))\
    .withColumn('cd_eqpt', df_eventlog["cd_eqpt"].cast(StringType()))\
    .withColumn('nr_idef_secu', df_eventlog["nr_idef_secu"].cast(StringType())) \
    .withColumn('dh_even', df_eventlog["dh_even"].cast(TimestampType()))\
    .withColumn('nr_eventos', df_eventlog["nr_eventos"].cast(IntegerType()))

df_eventlog.printSchema()   

root
 |-- nr_idef_secu: string (nullable = true)
 |-- cd_eqpt: string (nullable = true)
 |-- nr_sequ_even: string (nullable = true)
 |-- dh_even: timestamp (nullable = true)
 |-- nr_eventos: integer (nullable = false)



In [11]:
# obtem a data mais recente do log de eventos de (2015, 7, 8) a (2016, 12, 1)
# DESCOBERTAS DATAS DO FUTURO NO LOG??? 01-12-2016)
#df_eventlog.orderBy(col('dh_even').desc()).limit(10).toPandas()

In [12]:
#df_eventlog.limit(10).orderBy(col("nr_eventos").desc()).toPandas()

In [13]:
# DEPOIS: acertar abaixo para pegar direto ultima data do arquivo original como data_ref
# Filtrando últimos 90 dias
dias_a_subtrair = 90 
import datetime, time
from dateutil.relativedelta import relativedelta

# obtendo faixa do período de histórico a partir da data de referencia
data_ref = datetime.datetime(2016, 8, 8) 
noventa_dias_antes = data_ref + relativedelta(days=-dias_a_subtrair)
print (noventa_dias_antes)
type (noventa_dias_antes)

2016-05-10 00:00:00


datetime.datetime

In [14]:
data_atual = ("2016-08-08")
data_ref = ("2016-05-10")
dt_sql_atual = (time.mktime(datetime.datetime.strptime(data_atual, "%Y-%m-%d").timetuple()))
dt_sql = (time.mktime(datetime.datetime.strptime(data_ref, "%Y-%m-%d").timetuple()))
dt_sql

1462849200.0

In [15]:
# FILTRANDO SOMENTE ULTIMOS 90 DIAS
#df_eventlog = df_eventlog.where("CAST(dh_even AS INT) >= {0}".format(dt_sql)).orderBy("dh_even")
df_eventlog = df_eventlog.where("CAST(dh_even AS INT) BETWEEN {0} AND {1}"\
                                .format(dt_sql,dt_sql_atual)).orderBy("dh_even")
df_eventlog.take(2)

[Row(nr_idef_secu=u'105', cd_eqpt=u'171', nr_sequ_even=u'7023', dh_even=datetime.datetime(2016, 5, 10, 0, 0), nr_eventos=1),
 Row(nr_idef_secu=u'105', cd_eqpt=u'171', nr_sequ_even=u'7023', dh_even=datetime.datetime(2016, 5, 10, 0, 0), nr_eventos=1)]

In [183]:
# MUDAR:
# 1. Separar aqui
(trainingData_rf, testData_rf) = df_eventlog.randomSplit([0.7, 0.3])

## Preparando Dados de Treino e de Teste

In [184]:
trainingData_rf.count()

7730040

## Obtendo Eventos Marcadores de POC (selecionados como features)
Usando como baseline o arquivo de POCs reportados, serão identificados os eventos fora do padrão das agências/ATMs para serem usados como marcadores de POC no modelo preditivo.
São considerados fora do padrão, eventos que ocorrem em dias de ocorrência de POC, em agências reportadas como tendo ocorrência de POC, e que têm a quantidade de ocorrências no dia acima do percentil 75% da distribuição diária do eventos em todas as ATMs. Além disso, devem ser eventos que não aparecem com valores maiores que o percentil 75% nos dias em que não existem ocorrências de POCs.

In [18]:
# REPETIR PARA DADOS DE TESTE
# DUVIDA: tem sentido pegar o percentil e mediana de eventos como um todo ao inves de por Agencia/ATM como faço?
# Por enquanto manter por agencia/atm

# calculando percentis de baseline para todas as observações do log com base em agencia/atm/evento
# OBS: o uso do percentil desta forma permite usar os eventos isoladamente como marcadores 
# pois normaliza-os, tornando este marcador independente das ATM onde ocorrem
df_eventlog_percen = trainingData_rf.withColumn('id', concat(col("nr_idef_secu"), lit("-"), 
                                             col("cd_eqpt"), lit("-"),col("nr_sequ_even")))\
                                .select(['id', 'nr_eventos'])

In [19]:
#df_eventlog_percen.take(5)

In [20]:
# REPETIR PARA DADOS DE TESTE
mediana = 50
percentil_max = 75
rdd_eventlog_percen = df_eventlog_percen.rdd \
    .map(lambda x: (x[0], (x[1],), )) \
    .reduceByKey(lambda acc, val: acc + val) \
    .map(lambda x: (x[0], np.percentile(x[1], mediana), np.percentile(x[1], percentil_max)))

rdd_eventlog_percen = rdd_eventlog_percen \
    .map(lambda k: (k[0].split("-"), k[1], k[2])) 

df_eventlog_percen = rdd_eventlog_percen \
    .map(lambda p: (p[0][0],p[0][1],p[0][2],float(p[1]), float(p[2])  )) \
    .toDF(['nr_idef_secu',
           'cd_eqpt',
           'nr_sequ_even',
           'mediana',
           'percentil_max'
                   ])
#df_eventlog_percen.limit(10).toPandas()

df_eventlog_percen.printSchema()

root
 |-- nr_idef_secu: string (nullable = true)
 |-- cd_eqpt: string (nullable = true)
 |-- nr_sequ_even: string (nullable = true)
 |-- mediana: double (nullable = true)
 |-- percentil_max: double (nullable = true)



In [21]:
#df_eventlog_percen.count()

In [22]:
#df_eventlog_percen.orderBy(col('percentil_max').desc()).limit(10).toPandas()

In [23]:
#df_eventlog_percen.orderBy(col('percentil_max').desc()).filter(col('percentil_max') > col('mediana')).limit(10).toPandas()

In [24]:
# REPETIR PARA DADOS DE TESTE
# colocar no log total as colunas de mediana e percentil máximo por agencia+atm+evento 
df_eventlog2 = trainingData_rf \
    .join(df_eventlog_percen, (trainingData_rf['nr_idef_secu'] == df_eventlog_percen['nr_idef_secu']) &
                              (trainingData_rf['cd_eqpt'] == df_eventlog_percen['cd_eqpt']) &
                              (trainingData_rf['nr_sequ_even'] == df_eventlog_percen['nr_sequ_even']), 'left') \
    .drop(df_eventlog_percen["nr_idef_secu"])\
    .drop(df_eventlog_percen["cd_eqpt"])\
    .drop(df_eventlog_percen["nr_sequ_even"])

#df_eventlog2.count()

# tirando zeros a esquerda e truncando codigos de agencia e atm
df_eventlog2 = df_eventlog2\
    .withColumn('cd_eqpt', trim(df_eventlog2.cd_eqpt))\
    .withColumn('nr_idef_secu', trim(df_eventlog2.nr_idef_secu))\
    .withColumn('nr_idef_secu', regexp_replace('nr_idef_secu', '^0+(?!$)', '')) # removendo zero `esquerda
#    .withColumn('nr_idef_secu', regexp_replace('nr_idef_secu', '^0', '')) # removendo zero `esquerda
# testar s.replaceFirst("^0+(?!$)", "")

In [25]:
#df_eventlog2.orderBy(col('mediana').desc()).limit(10).toPandas()

### Dados de registro de POC
Obtém os POCs de Retenção de Cartão, somente para os últimos 90 dias.
Trata as seguintes colunas:
+ data de cadastro do POC
+ código da ATM onde ocorreu o POC
+ código da agência onde ocorreu o POC

Obtém os POCs de Golpe reportados.
Identifica quais eventos que aconteceram nas agências e na data que saíram da faixa de percentil. estes eventos serão os **Marcadores de POC**.

In [26]:
rdd_poc = sc.textFile(poc_in)
print ("\n\nNúmero de linhas carregadas: " + str("%d" % (rdd_poc.count())))

header = rdd_poc.first()
df_poc = rdd_poc \
    .filter(lambda line: line != header) \
    .map(lambda k: k.split(",")) \
    .map(lambda p: (p[0],p[2],p[4], p[8])) \
    .toDF(['data_cad', 
            'atm',
            'agencia',
            'golpe'
        ])

df_poc = df_poc.withColumn('data_cad', funcStrToDate_dmyyyy(col('data_cad'))) 

df_poc = df_poc \
    .withColumn('atm', df_poc["atm"].cast(StringType()))\
    .withColumn('agencia', df_poc["agencia"].cast(StringType())) \
    .withColumn('data_cad', df_poc["data_cad"].cast(TimestampType())) \
    .withColumn('golpe', df_poc["golpe"].cast(StringType())) \

# FILTRANDO POCS SOMENTE DOS ULTIMOS 90 DIAS E SOMENTE PARA GOLPE DE RETENCAO DE CARTAO
#df_poc = df_poc.where("golpe = 'GOLPE RETENTOR DE CARTAO' and \
#            CAST(data_cad AS INT) >= {0}".format(dt_sql)).orderBy('agencia', 'data_cad')
df_poc = df_poc.where("CAST(data_cad AS INT) >= {0}".format(dt_sql)).orderBy('agencia', 'data_cad')



Número de linhas carregadas: 1019


In [27]:
# tirando zeros a esquerda e truncando codigos de agencia e atm
df_poc = df_poc\
    .withColumn('atm', trim(df_poc.atm))\
    .withColumn('agencia', trim(df_poc.agencia))\
    .withColumn('agencia', regexp_replace('agencia', '^0+(?!$)', '')) # removendo zero `esquerda
#df_poc.limit(10).toPandas()

In [28]:
df_poc.orderBy(col('data_cad').desc()).toPandas()

data_cad   atm     agencia                       golpe
0   2016-08-08               140  GOLPE RETENTOR DE ENVELOPE
1   2016-08-08               985  GOLPE RETENTOR DE ENVELOPE
2   2016-08-08                59  GOLPE RETENTOR DE ENVELOPE
3   2016-08-08               112    GOLPE RETENTOR DE CARTAO
4   2016-08-08               204  GOLPE RETENTOR DE ENVELOPE
5   2016-08-08               159  GOLPE RETENTOR DE ENVELOPE
6   2016-08-08              3547  GOLPE RETENTOR DE ENVELOPE
7   2016-08-08               915  GOLPE RETENTOR DE ENVELOPE
8   2016-08-08              2065  GOLPE RETENTOR DE ENVELOPE
9   2016-08-08              4281  GOLPE RETENTOR DE ENVELOPE
10  2016-08-08              4550    GOLPE RETENTOR DE CARTAO
11  2016-08-08              4550    GOLPE RETENTOR DE CARTAO
12  2016-08-08               142  GOLPE RETENTOR DE ENVELOPE
13  2016-08-08  "151         153                        3316
14  2016-08-08  "155         157                  167 e 168"
15  2016-08-08  "151  158 e 162"          SEGURANÇA 24 HORAS
16  2016-08-08              3940  GOLPE RETENTOR DE ENVELOPE
17  2016-08-08              4268    GOLPE RETENTOR DE CARTAO
18  2016-08-06               995  GOLPE RETENTOR DE ENVELOPE
19  2016-08-06              4776  GOLPE RETENTOR DE ENVELOPE
20  2016-08-06              1074              TELEFONE FALSO
21  2016-08-06              3898  GOLPE RETENTOR DE ENVELOPE
22  2016-08-06              1561    GOLPE RETENTOR DE CARTAO
23  2016-08-06              4263  GOLPE RETENTOR DE ENVELOPE
24  2016-08-06               935              TELEFONE FALSO
25  2016-08-06              4268    GOLPE RETENTOR DE CARTAO
26  2016-08-04     0         787  GOLPE RETENTOR DE ENVELOPE
27  2016-08-04     0         787    GOLPE RETENTOR DE CARTAO
28  2016-08-04     0        3989  GOLPE RETENTOR DE ENVELOPE
29  2016-08-04     0        3146    GOLPE RETENTOR DE CARTAO
..         ...   ...         ...                         ...
354 2016-05-16              2082    GOLPE RETENTOR DE CARTAO
355 2016-05-16              4433                            
356 2016-05-16              1207  GOLPE RETENTOR DE ENVELOPE
357 2016-05-16               194  GOLPE RETENTOR DE ENVELOPE
358 2016-05-16              3717  GOLPE RETENTOR DE ENVELOPE
359 2016-05-16              4395         LEITORAS OBSTRUIDAS
360 2016-05-16               959  GOLPE RETENTOR DE ENVELOPE
361 2016-05-16              1227    GOLPE RETENTOR DE CARTAO
362 2016-05-16              3982         LEITORAS OBSTRUIDAS
363 2016-05-14               277  GOLPE RETENTOR DE ENVELOPE
364 2016-05-14              3987         TECLADOS OBSTRUÍDOS
365 2016-05-14              4433         LEITORAS OBSTRUIDAS
366 2016-05-14               340  GOLPE RETENTOR DE ENVELOPE
367 2016-05-14               264                         POC
368 2016-05-14               570         TECLADOS OBSTRUÍDOS
369 2016-05-14              1093  GOLPE RETENTOR DE ENVELOPE
370 2016-05-14                86                         POC
371 2016-05-14               259  GOLPE RETENTOR DE ENVELOPE
372 2016-05-13               909  GOLPE RETENTOR DE ENVELOPE
373 2016-05-13              3066                         POC
374 2016-05-13              2286                         POC
375 2016-05-12              1641                         POC
376 2016-05-12              2286                         POC
377 2016-05-11              4279  GOLPE RETENTOR DE ENVELOPE
378 2016-05-11               114  GOLPE RETENTOR DE ENVELOPE
379 2016-05-11              4425         LEITORAS OBSTRUIDAS
380 2016-05-11              3733         LEITORAS OBSTRUIDAS
381 2016-05-11              4424              TELEFONE FALSO
382 2016-05-10               245                         POC
383 2016-05-10              3385                         POC

[384 rows x 4 columns]

In [29]:
#df_eventlog2.printSchema()

In [30]:
# DESNECESSARIA A PESQUISA ABAIXO AGORA

#df_eventlog2.registerTempTable("tb_logeven")
#df_poc.registerTempTable("tb_pocagen")

#df_evenpoc = sqlContext.sql("select * from tb_logeven where exists \
#                        (select agencia from tb_pocagen where tb_logeven.nr_idef_secu = tb_pocagen.agencia)")

# TESTAR AQUI SE SELECIONOU SOMENTE AGENCIAS DO POC
# tem que trazer igual ou menor ao numero de agencias do df_poc
#df_evenpoc.groupBy('nr_idef_secu').count().orderBy('nr_idef_secu').toPandas()

## Selecionando Eventos por ATM/Agência em dias POC

In [31]:
# SELECIONANDO DO LOG TOTAL SOMENTE AGENCIAS E DIAS DE OCORRENCIA DE POC ACIMA DO PERCENTIL
df_poceven = df_eventlog2 \
    .join(df_poc, (df_poc['data_cad'] == df_eventlog2['dh_even']) &
                (df_poc['agencia'] == df_eventlog2['nr_idef_secu']), 'left')
#df_poceven.limit(10).toPandas()

# LOG SOMENTE COM AGENCIAS POC NAS DATAS POC E MAIOR QUE PERCENTIL
df_poceven2 = df_poceven\
    .filter((col("nr_eventos") > col('percentil_max')) &
            ((col("agencia") == col('nr_idef_secu'))))

#df_poceven2.limit(10).toPandas()

## Selecionando Eventos por ATM/Agência Fora de dias POC

In [32]:
# LOG SOMENTE COM TODAS AS AGENCIAS NAS DATAS FORA DE POC E MENOR QUE PERCENTIL
df_pocevenNAO = df_poceven\
    .filter((col("nr_eventos") <= col('percentil_max')) &
            (col('agencia').isNull()))
#    .filter(col('agencia').isNull())
#df_pocevenNAO.limit(10).toPandas()

# JOIN DAS DUAS ACIMA POR EVENTO PARA OBTER LISTA DE POKEVENTOS
# DEVE SER LEFT PARA PEGAR EVENTOS POC QUE NAO OCORREM FORA DO POC
df_pokeventos = df_poceven2\
    .join(df_pocevenNAO,
          (df_poceven2['nr_sequ_even'] == df_pocevenNAO['nr_sequ_even']), 'left')\
    .drop(df_pocevenNAO["nr_sequ_even"])\
    .groupBy('nr_sequ_even').count()
    
#df_pokeventos.limit(10).toPandas()
#    .drop(df_eventlog_percen["nr_idef_secu"])\
#    .drop(df_eventlog_percen["cd_eqpt"])\
#    .drop(df_eventlog_percen["nr_sequ_even"])

In [33]:
# CHECK
#df_pokeventos.limit(10).toPandas()

In [34]:
# LISTA DE EVENTOS QUE APARECEM COM VALOR MAIOR QUE PERCENTIL EN DIA DE POC E
# VALOR ABAIXO DO PERCENTIL EM DIAS FORA DE POC
l_evenpoc = df_pokeventos.toPandas().nr_sequ_even.tolist()
l_evenpoc

[u'5603', u'5610', u'5612', u'5616', u'5620', u'5622', u'5640']

## Preparando dados processados
Marcando como POC as entradas do log para os eventos marcadores 

In [35]:
# AQUI VOLTAR A FAZER COMO NO V7. MAS MANTER A SEPERACAO TREINO E TESTE COMO FEZ ACIMA

In [36]:
# APROXIMACAO ABAIXO PARA EVITAR PROBLEMAS NO withColumns
# criar coluna de indice de proximidade da mediana (iproxmed)
# calculado como número de ocorrências real diaria/mediana diaria de eventos por agencia e atm
# If you use deimcal, you need to import

#from decimal import getcontext, Decimal
# Set the precision.
#getcontext().prec = 3

#format(math.pi, '.2f')
# mudar o arredondamento abaixo para ver se afeta o modelo
df_feat = df_eventlog2\
    .withColumn('iproxmed', (col("nr_eventos")/col("mediana"))*1000.0)
#    .withColumn('iproxmed', round(col("nr_eventos")/col("mediana"),2))
    #df_feat.limit(10).toPandas()

df_feat.printSchema()

# transpondo tipo de evento para gerar features
df_feat = df_feat\
    .drop(df_feat["mediana"])

#df_feat.groupBy(['nr_idef_secu']).count().toPandas()

# FILTRANDO SOMENTE FEATURES DE POKEVENTOS
df_feat = df_feat.filter(col("nr_sequ_even").isin(l_evenpoc))

df_feat.printSchema()

# OBS: selecionar somente os eventos usados no mllib como features para usar menos memoria:
pd_pivot = df_feat\
    .select('dh_even', 'nr_idef_secu', 'cd_eqpt', 'nr_sequ_even', 'iproxmed').toPandas()

root
 |-- nr_idef_secu: string (nullable = true)
 |-- cd_eqpt: string (nullable = true)
 |-- nr_sequ_even: string (nullable = true)
 |-- dh_even: timestamp (nullable = true)
 |-- nr_eventos: integer (nullable = false)
 |-- mediana: double (nullable = true)
 |-- percentil_max: double (nullable = true)
 |-- iproxmed: double (nullable = true)

root
 |-- nr_idef_secu: string (nullable = true)
 |-- cd_eqpt: string (nullable = true)
 |-- nr_sequ_even: string (nullable = true)
 |-- dh_even: timestamp (nullable = true)
 |-- nr_eventos: integer (nullable = false)
 |-- percentil_max: double (nullable = true)
 |-- iproxmed: double (nullable = true)



In [37]:
#pd_pivot

In [38]:
# transpondo evento por tipo OK! 
# OBS: a funcao .reset_index abaixo é necessário para evitar a mesclagem das células fixas
pd_feat = pd.pivot_table(pd_pivot,index=["nr_idef_secu","cd_eqpt","dh_even"],
                         columns=["nr_sequ_even"], margins=False, values=["iproxmed"], fill_value=0).reset_index()
# acertando nomes das colunas geradas em linhas diferentes

pd_feat.columns = [c[1] if c[1] else c[0] for c in pd_feat.columns.tolist()]


# criar coluna com data como string para posterior transformação para dataframe
pd_feat = pd_feat.assign(data_even = lambda x: x['dh_even'].dt.strftime('%Y-%m-%d') ).drop('dh_even', axis=1)
pd_feat

# salva como csv pois conversao para Dataframe Spark apresenta erro. provavel falta de memoria
#pd_feat.to_csv(tabelao_out)

df_feat_alt = sqlCtx.createDataFrame(pd_feat)
# FALTA converter data em timedate de novo (converter para string no Pandas antes)

df_feat_alt.take(5)

[Row(nr_idef_secu=u'3063', cd_eqpt=u'10001', 5603=1000, 5610=1000, 5612=1000, 5616=1000, 5620=1000, 5622=1000, 5640=1000, data_even=u'2016-06-01'),
 Row(nr_idef_secu=u'3063', cd_eqpt=u'10001', 5603=1000, 5610=0, 5612=1000, 5616=1000, 5620=1000, 5622=0, 5640=1000, data_even=u'2016-06-02'),
 Row(nr_idef_secu=u'3063', cd_eqpt=u'10001', 5603=1000, 5610=0, 5612=0, 5616=1000, 5620=0, 5622=0, 5640=1000, data_even=u'2016-06-03'),
 Row(nr_idef_secu=u'3063', cd_eqpt=u'10001', 5603=1000, 5610=1000, 5612=0, 5616=1000, 5620=0, 5622=1000, 5640=0, data_even=u'2016-06-04'),
 Row(nr_idef_secu=u'3063', cd_eqpt=u'10001', 5603=1000, 5610=1000, 5612=1000, 5616=1000, 5620=0, 5622=0, 5640=1000, data_even=u'2016-06-05')]

In [40]:
df_feat_alt = df_feat_alt\
   .withColumn('5603', df_feat_alt["5603"].cast(DoubleType()))\
    .withColumn('5610', df_feat_alt["5610"].cast(DoubleType()))\
    .withColumn('5612', df_feat_alt["5612"].cast(DoubleType()))\
    .withColumn('5616', df_feat_alt["5616"].cast(DoubleType()))\
    .withColumn('5620', df_feat_alt["5620"].cast(DoubleType()))\
    .withColumn('5622', df_feat_alt["5622"].cast(DoubleType()))\
    .withColumn('5640', df_feat_alt["5640"].cast(DoubleType()))

In [41]:
df_feat_alt.printSchema()

root
 |-- nr_idef_secu: string (nullable = true)
 |-- cd_eqpt: string (nullable = true)
 |-- 5603: double (nullable = true)
 |-- 5610: double (nullable = true)
 |-- 5612: double (nullable = true)
 |-- 5616: double (nullable = true)
 |-- 5620: double (nullable = true)
 |-- 5622: double (nullable = true)
 |-- 5640: double (nullable = true)
 |-- data_even: string (nullable = true)



In [42]:
# OBS: se já tiver o tabelao_alt gerado, pode rodar a partir daqui!!!!!
#############################################################################
# AQUI TIVE QUE FAZER ESTA GAMBIARRA: salvar como csv e alguma manipulaçao (acertar cabeçalho) para depois ler
# até resolver o problema de memoria gerado por fazer o pivot no Pandas
#rdd_feat_alt = sc.textFile(tabelao_in)

In [43]:
#rdd_feat_alt.first()
#df = sqlContext.load(source="com.databricks.spark.csv", header="true", path = "cars.csv")

In [44]:
#print ("\n\nNúmero de linhas carregadas: " + str("%d" % (rdd_feat_alt.count())))

In [45]:
#header = rdd_feat_alt.first()
#chemaString = header
#fields = [StructField(field_name, StringType(), False) for field_name in schemaString.split(',')]
#schema = StructType(fields)

#rdd_feat_alt = rdd_feat_alt \
#    .filter(lambda line: line != header) \
#    .map(lambda k: k.split(","))

#df_feat_alt = sqlContext.createDataFrame(rdd_feat_alt, schema)

In [46]:
#df_feat_alt.limit(10).toPandas()

In [47]:
#df_feat_alt.printSchema()

In [48]:
df_feat_alt = df_feat_alt.withColumn('data_even', df_feat_alt["data_even"].cast(TimestampType()))
#df_feat_alt.limit(10).toPandas()

In [49]:
df_feat_alt = df_feat_alt\
    .withColumn('nr_idef_secu', trim(df_feat_alt.nr_idef_secu))
#df_feat_alt.printSchema()

In [50]:
#df_feat_alt.printSchema()

In [143]:
# primeiro left_join df_feat_alt + df_poc
    # depois para agencia == cd_secu and cada coluna evento > 1 = POC
df_tidy = df_feat_alt \
    .join(df_poc, (df_feat_alt['nr_idef_secu'] == df_poc['agencia']) &
            (df_feat_alt['data_even'] == df_poc['data_cad']), 'left')
#df_tidy.limit(10).toPandas()

In [144]:
df_tidy.orderBy(col('agencia').desc())\
    .filter(col('agencia') == col('nr_idef_secu'))\
    .select('agencia').distinct().toPandas()
# 53 agências distintas com POC

agencia
0     787
1    3350
2    3063
3    1217
4    1053

In [145]:
df_tidy.printSchema()

root
 |-- nr_idef_secu: string (nullable = true)
 |-- cd_eqpt: string (nullable = true)
 |-- 5603: long (nullable = true)
 |-- 5610: long (nullable = true)
 |-- 5612: long (nullable = true)
 |-- 5614: long (nullable = true)
 |-- 5616: long (nullable = true)
 |-- 5620: long (nullable = true)
 |-- 5622: long (nullable = true)
 |-- 5640: long (nullable = true)
 |-- data_even: timestamp (nullable = true)
 |-- data_cad: timestamp (nullable = true)
 |-- atm: string (nullable = true)
 |-- agencia: string (nullable = true)
 |-- golpe: string (nullable = true)



In [53]:
#df_tidy.limit(10).toPandas()

In [146]:
# Transformar em double as features (features selecionadas) (ERRO)
for even in l_evenpoc:
    df_tidy_dbl = df_tidy.withColumn(even, df_tidy[even].cast(DoubleType()))

In [147]:
# TESTAR PORQUE NAO FUNCIONA LOOP ABAIXO
#df_tidy_dbl.printSchema()

root
 |-- nr_idef_secu: string (nullable = true)
 |-- cd_eqpt: string (nullable = true)
 |-- 5603: long (nullable = true)
 |-- 5610: long (nullable = true)
 |-- 5612: long (nullable = true)
 |-- 5614: long (nullable = true)
 |-- 5616: long (nullable = true)
 |-- 5620: long (nullable = true)
 |-- 5622: long (nullable = true)
 |-- 5640: double (nullable = true)
 |-- data_even: timestamp (nullable = true)
 |-- data_cad: timestamp (nullable = true)
 |-- atm: string (nullable = true)
 |-- agencia: string (nullable = true)
 |-- golpe: string (nullable = true)



In [149]:
df_tidy_dbl = df_tidy\
    .withColumn('5603', df_tidy['5603'].cast(DoubleType()))\
    .withColumn('5610', df_tidy['5610'].cast(DoubleType()))\
    .withColumn('5612', df_tidy['5612'].cast(DoubleType()))\
    .withColumn('5614', df_tidy['5614'].cast(DoubleType()))\
    .withColumn('5616', df_tidy['5616'].cast(DoubleType()))\
    .withColumn('5620', df_tidy['5620'].cast(DoubleType()))\
    .withColumn('5622', df_tidy['5622'].cast(DoubleType()))\
    .withColumn('5640', df_tidy['5640'].cast(DoubleType()))

In [152]:
df_tidy_dbl.show()

+------------+-------+------+------+------+------+------+------+------+------+--------------------+--------+----+-------+-----+
|nr_idef_secu|cd_eqpt|  5603|  5610|  5612|  5614|  5616|  5620|  5622|  5640|           data_even|data_cad| atm|agencia|golpe|
+------------+-------+------+------+------+------+------+------+------+------+--------------------+--------+----+-------+-----+
|        1053|  10001|1000.0|1000.0|1000.0|   0.0|1000.0|   0.0|1000.0|   0.0|2016-06-22 00:00:...|    null|null|   null| null|
|        1053|  10001|1000.0|   0.0|1000.0|1000.0|1000.0|1000.0|1000.0|1000.0|2016-06-24 00:00:...|    null|null|   null| null|
|        1217|  10001|1000.0|   0.0|1000.0|1000.0|1000.0|1000.0|   0.0|1000.0|2016-07-15 00:00:...|    null|null|   null| null|
|        1523|  10001|1000.0|1000.0|   0.0|1000.0|1000.0|1000.0|1000.0|1000.0|2016-05-22 00:00:...|    null|null|   null| null|
|        3350|  10001|1500.0|2000.0|   0.0|2000.0|1000.0|1000.0|2000.0|2000.0|2016-06-11 00:00:...|    n

In [153]:
# MARCA POC PARA AGENCIAS QUE OCORREM NA TABELA POC, NA DATA POC E QUE AO MENOS UM EVENTO(FEATURE)
# TENHA VALOR MAIOR QUE 1 (INDICANDO OCORRENCIA ACIMA DA MEDIANA)

##############################################################################
# POSSIVEL MELHORIA EM FASE 2: ao inves de label, criar target para regressao, com valor igual
# ao maximo do valor de quantos eventos ocorrem acima de 1 nas features
##############################################################################
# antes separar somente as colunas de eventos
from functools import reduce
def superMax(*cols):
    return reduce(lambda a, b: __builtin__.max(a,b) ,cols)

add = udf(superMax)

df_tidy_dbl = df_tidy_dbl.withColumn('max_evento', add(*[df_tidy_dbl[x] for x in df_tidy_dbl.columns[3:-8]]))
# tentar cast par double acima    

In [154]:
df_tidy_dbl.printSchema()

root
 |-- nr_idef_secu: string (nullable = true)
 |-- cd_eqpt: string (nullable = true)
 |-- 5603: double (nullable = true)
 |-- 5610: double (nullable = true)
 |-- 5612: double (nullable = true)
 |-- 5614: double (nullable = true)
 |-- 5616: double (nullable = true)
 |-- 5620: double (nullable = true)
 |-- 5622: double (nullable = true)
 |-- 5640: double (nullable = true)
 |-- data_even: timestamp (nullable = true)
 |-- data_cad: timestamp (nullable = true)
 |-- atm: string (nullable = true)
 |-- agencia: string (nullable = true)
 |-- golpe: string (nullable = true)
 |-- max_evento: string (nullable = true)



In [155]:
# POSSIVEL PROBLEMA NA CONVERSAO DE STRING PARA DOUBLE NO USO DE withColumn
df_tidy_dbl = df_tidy_dbl.withColumn('max_evento', df_tidy_dbl["max_evento"].cast(DoubleType()))

In [156]:
df_tidy_dbl.take(2)

[Row(nr_idef_secu=u'1053', cd_eqpt=u'10001', 5603=1000.0, 5610=1000.0, 5612=1000.0, 5614=1000.0, 5616=1000.0, 5620=0.0, 5622=1000.0, 5640=1000.0, data_even=datetime.datetime(2016, 6, 7, 0, 0), data_cad=None, atm=None, agencia=None, golpe=None, max_evento=1000.0),
 Row(nr_idef_secu=u'1217', cd_eqpt=u'10001', 5603=1000.0, 5610=0.0, 5612=0.0, 5614=1000.0, 5616=1000.0, 5620=1000.0, 5622=1000.0, 5640=1000.0, data_even=datetime.datetime(2016, 5, 24, 0, 0), data_cad=None, atm=None, agencia=None, golpe=None, max_evento=1000.0)]

In [157]:
# TENTATIVA 1 (FUCIONOU COM df_tidy!!????) NAO funciona com comparacoa com double no MAC??? versao spark
df_tidy_dbl = df_tidy_dbl.withColumn("label",
    when((df_tidy_dbl["max_evento"] > 1000.0), 1.0)\
        .otherwise(0.0))

In [158]:
df_tidy_dbl.printSchema()

root
 |-- nr_idef_secu: string (nullable = true)
 |-- cd_eqpt: string (nullable = true)
 |-- 5603: double (nullable = true)
 |-- 5610: double (nullable = true)
 |-- 5612: double (nullable = true)
 |-- 5614: double (nullable = true)
 |-- 5616: double (nullable = true)
 |-- 5620: double (nullable = true)
 |-- 5622: double (nullable = true)
 |-- 5640: double (nullable = true)
 |-- data_even: timestamp (nullable = true)
 |-- data_cad: timestamp (nullable = true)
 |-- atm: string (nullable = true)
 |-- agencia: string (nullable = true)
 |-- golpe: string (nullable = true)
 |-- max_evento: double (nullable = true)
 |-- label: double (nullable = false)



In [159]:
df_tidy_dbl.groupBy("label").count().orderBy("label").show()

+-----+-----+
|label|count|
+-----+-----+
|  0.0|  815|
|  1.0|   12|
+-----+-----+



In [160]:
# tabelao com labels
#df_tidy.toPandas().to_csv(tabelao_label)

In [161]:
#df_tidy.printSchema()

In [162]:
# ATANCAO: ver se falta fazer antes
#df_tidy = df_feat_alt\
#   .withColumn('data_even', df_feat_alt["data_even"].cast(TimestampType()))\
#    .withColumn('label', df_feat_alt["label"].cast(DoubleType()))

In [163]:
#df_tidy.limit(10).toPandas()

In [164]:
#df_tidy.printSchema()

In [165]:
# salvando tabelao com label
#df_tidy.toPandas().to_csv(tabelao_final)

## Modelagem
O modelo usado será o de Random Forest, implementando pipelines da library ML do Spark.
A partir da amostra de dados será selecionado um subset com 75% dos dados para treinar o modelo. 

### Treino do Modelo
Prepara a amostra de acordo com o formato de entrada de label e vetor de features esperados pelo modelo.

In [166]:
from pyspark.mllib.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [167]:
# Transformar em double as features (features selecionadas)
# por algum motivo nao esta rodando no loop e nao á erro!!??
#for even in l_evenpoc:
#    df_tidy_dbl = df_tidy\
#        .withColumn(even, df_tidy[even].cast(DoubleType()))

df_tidy_dbl.printSchema()

root
 |-- nr_idef_secu: string (nullable = true)
 |-- cd_eqpt: string (nullable = true)
 |-- 5603: double (nullable = true)
 |-- 5610: double (nullable = true)
 |-- 5612: double (nullable = true)
 |-- 5614: double (nullable = true)
 |-- 5616: double (nullable = true)
 |-- 5620: double (nullable = true)
 |-- 5622: double (nullable = true)
 |-- 5640: double (nullable = true)
 |-- data_even: timestamp (nullable = true)
 |-- data_cad: timestamp (nullable = true)
 |-- atm: string (nullable = true)
 |-- agencia: string (nullable = true)
 |-- golpe: string (nullable = true)
 |-- max_evento: double (nullable = true)
 |-- label: double (nullable = false)



In [168]:
# rodando para as features com mais relevância já analisadas
#assemblerTrain = VectorAssembler(
#    inputCols=["655", "689", "1615", "1620", "1653", "2032", "2034", "2422", "2699", "2846",
#                "3009", "3010", "3011", "3321", "3367"],
#   outputCol="features")
assemblerTrain = VectorAssembler(
    inputCols=l_evenpoc,
   outputCol="features")

#               ,"1617", "1689","2433", "1746", "1568","883", "2163"],
 
df_tmp = assemblerTrain.transform(df_tidy_dbl)
# TESTAR ABAIXO SELECIONAR AGENCIA + ATM
df_tidy_rf = df_tmp.select("label", "features")
df_tidy_rf.printSchema()

root
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)



In [174]:
#df_tmp.printSchema()

In [169]:
df_tidy_rf.take(20)

[Row(label=0.0, features=DenseVector([1000.0, 0.0, 1000.0, 0.0, 1000.0, 1000.0, 0.0, 1000.0])),
 Row(label=0.0, features=DenseVector([1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 0.0])),
 Row(label=0.0, features=DenseVector([1000.0, 1000.0, 0.0, 0.0, 1000.0, 1000.0, 1000.0, 1000.0])),
 Row(label=0.0, features=DenseVector([1000.0, 1000.0, 1000.0, 1000.0, 0.0, 1000.0, 1000.0, 1000.0])),
 Row(label=0.0, features=SparseVector(8, {0: 1000.0})),
 Row(label=0.0, features=SparseVector(8, {1: 1000.0, 2: 1000.0, 6: 1000.0, 7: 1000.0})),
 Row(label=0.0, features=SparseVector(8, {0: 1000.0, 1: 1000.0, 4: 1000.0, 7: 1000.0})),
 Row(label=0.0, features=DenseVector([1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0])),
 Row(label=0.0, features=DenseVector([1000.0, 1000.0, 1000.0, 0.0, 1000.0, 1000.0, 1000.0, 1000.0])),
 Row(label=0.0, features=DenseVector([1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0])),
 Row(label=0.0, features=SparseVector(8, {0: 1000.0, 1:

In [170]:
# Split the data into training and test sets (30% held out for testing)
#(trainingData_rf, testData_rf) = df_tidy_rf.randomSplit([0.7, 0.3])

In [171]:
# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(df_tidy_rf)
#labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(trainingData_rf)

In [175]:
# Automatically identify categorical features, and index them.
# We specify maxCategories so features with > 2 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=2).fit(df_tidy_rf)
#featureIndexer =\
#    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=2).fit(trainingData_rf)

In [176]:
#trainingData_rf.groupBy("label").count().orderBy("label").show()

In [177]:
#testData_rf.groupBy("label").count().orderBy("label").show()

In [178]:
# Train a Random Forest model.
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures")
#dt = DecisionTreeClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures")

In [179]:
# Chain indexers and tree in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf])

In [180]:
model = pipeline.fit(df_tidy_rf)

In [79]:
#pipeline = Pipeline(stages=[rf])
# Search through decision tree's maxDepth parameter for best model
#from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
#from pyspark.ml.evaluation import BinaryClassificationEvaluator
#paramGrid = ParamGridBuilder().addGrid(rf.maxDepth, [2,3,4,5,6,7]).build()

#numFolds = 5

In [80]:
#crossval = CrossValidator(
#    estimator=pipeline,
#    estimatorParamMaps=paramGrid,
#    evaluator=evaluator,
#    numFolds=numFolds)

In [81]:
# Fetch best model
#tree_model = model.bestModel.stages[2]
#print tree_model

In [82]:
# Train model.  This also runs the indexers.
#model = pipeline.fit(trainingData_rf)

## Persistência do Modelo
Salva o modelo para posterior uso. O código comentado abaixo deve ser descomentado nas manutenções para testar se modelo foi corretamente salvo.

In [83]:
# SAVE MODEL FOR FUTURE USE (TEST IF SAVED OK)
#model.save(sc, savedModel) # Erro. Funciona com mllib. Pesquisar como salvar pipeline

# Prepara Dados de Teste

In [185]:
# REPETIR PARA DADOS DE TESTE
df_eventlog_percen = testData_rf.withColumn('id', concat(col("nr_idef_secu"), lit("-"), 
                                             col("cd_eqpt"), lit("-"),col("nr_sequ_even")))\
                                .select(['id', 'nr_eventos'])

In [186]:
# REPETIR PARA DADOS DE TESTE
mediana = 50
percentil_max = 75
rdd_eventlog_percen = df_eventlog_percen.rdd \
    .map(lambda x: (x[0], (x[1],), )) \
    .reduceByKey(lambda acc, val: acc + val) \
    .map(lambda x: (x[0], np.percentile(x[1], mediana), np.percentile(x[1], percentil_max)))

rdd_eventlog_percen = rdd_eventlog_percen \
    .map(lambda k: (k[0].split("-"), k[1], k[2])) 

df_eventlog_percen = rdd_eventlog_percen \
    .map(lambda p: (p[0][0],p[0][1],p[0][2],float(p[1]), float(p[2])  )) \
    .toDF(['nr_idef_secu',
           'cd_eqpt',
           'nr_sequ_even',
           'mediana',
           'percentil_max'
                   ])
#df_eventlog_percen.limit(10).toPandas()

df_eventlog_percen.printSchema()

root
 |-- nr_idef_secu: string (nullable = true)
 |-- cd_eqpt: string (nullable = true)
 |-- nr_sequ_even: string (nullable = true)
 |-- mediana: double (nullable = true)
 |-- percentil_max: double (nullable = true)



In [194]:
# REPETIR PARA DADOS DE TESTE
# colocar no log total as colunas de mediana e percentil máximo por agencia+atm+evento 
df_eventlog2 = testData_rf \
    .join(df_eventlog_percen, (testData_rf['nr_idef_secu'] == df_eventlog_percen['nr_idef_secu']) &
                              (testData_rf['cd_eqpt'] == df_eventlog_percen['cd_eqpt']) &
                              (testData_rf['nr_sequ_even'] == df_eventlog_percen['nr_sequ_even']), 'left') \
    .drop(df_eventlog_percen["nr_idef_secu"])\
    .drop(df_eventlog_percen["cd_eqpt"])\
    .drop(df_eventlog_percen["nr_sequ_even"])

#df_eventlog2.count()

# tirando zeros a esquerda e truncando codigos de agencia e atm
df_eventlog2 = df_eventlog2\
    .withColumn('cd_eqpt', trim(df_eventlog2.cd_eqpt))\
    .withColumn('nr_idef_secu', trim(df_eventlog2.nr_idef_secu))\
    .withColumn('nr_idef_secu', regexp_replace('nr_idef_secu', '^0+(?!$)', '')) # removendo zero `esquerda
#    .withColumn('nr_idef_secu', regexp_replace('nr_idef_secu', '^0', '')) # removendo zero `esquerda
# testar s.replaceFirst("^0+(?!$)", "")

In [196]:
# SELECIONANDO DO LOG TOTAL SOMENTE AGENCIAS E DIAS DE OCORRENCIA DE POC ACIMA DO PERCENTIL
#df_poceven = df_eventlog2 \
#    .join(df_poc, (df_poc['data_cad'] == df_eventlog2['dh_even']) &
#                (df_poc['agencia'] == df_eventlog2['nr_idef_secu']), 'left')
#df_poceven.limit(10).toPandas()

# LOG SOMENTE COM AGENCIAS POC NAS DATAS POC E MAIOR QUE PERCENTIL
#df_poceven2 = df_poceven\
#    .filter((col("nr_eventos") > col('percentil_max')) &
#            ((col("agencia") == col('nr_idef_secu'))))

#df_poceven2.limit(10).toPandas()

In [197]:
# LOG SOMENTE COM TODAS AS AGENCIAS NAS DATAS FORA DE POC E MENOR QUE PERCENTIL
#df_pocevenNAO = df_poceven\
#    .filter((col("nr_eventos") <= col('percentil_max')) &
#            (col('agencia').isNull()))
#    .filter(col('agencia').isNull())
#df_pocevenNAO.limit(10).toPandas()

# JOIN DAS DUAS ACIMA POR EVENTO PARA OBTER LISTA DE POKEVENTOS
# DEVE SER LEFT PARA PEGAR EVENTOS POC QUE NAO OCORREM FORA DO POC
#df_pokeventos = df_poceven2\
#    .join(df_pocevenNAO,
#          (df_poceven2['nr_sequ_even'] == df_pocevenNAO['nr_sequ_even']), 'left')\
#    .drop(df_pocevenNAO["nr_sequ_even"])\
#    .groupBy('nr_sequ_even').count()

In [198]:
# LISTA DE EVENTOS QUE APARECEM COM VALOR MAIOR QUE PERCENTIL EN DIA DE POC E
# VALOR ABAIXO DO PERCENTIL EM DIAS FORA DE POC
#l_evenpoc = df_pokeventos.toPandas().nr_sequ_even.tolist()
#l_evenpoc

In [274]:
# APROXIMACAO ABAIXO PARA EVITAR PROBLEMAS NO withColumns
# criar coluna de indice de proximidade da mediana (iproxmed)
# calculado como número de ocorrências real diaria/mediana diaria de eventos por agencia e atm
# If you use deimcal, you need to import

#from decimal import getcontext, Decimal
# Set the precision.
#getcontext().prec = 3

#format(math.pi, '.2f')
# mudar o arredondamento abaixo para ver se afeta o modelo
df_feat = df_eventlog2\
    .withColumn('iproxmed', (col("nr_eventos")/col("mediana"))*1000.0)
#    .withColumn('iproxmed', round(col("nr_eventos")/col("mediana"),2))
    #df_feat.limit(10).toPandas()

df_feat.printSchema()

# transpondo tipo de evento para gerar features
df_feat = df_feat\
    .drop(df_feat["mediana"])

#df_feat.groupBy(['nr_idef_secu']).count().toPandas()

# FILTRANDO SOMENTE FEATURES DE POKEVENTOS
df_feat = df_feat.filter(col("nr_sequ_even").isin(l_evenpoc))

df_feat.printSchema()

# OBS: selecionar somente os eventos usados no mllib como features para usar menos memoria:
pd_pivot = df_feat\
    .select('dh_even', 'nr_idef_secu', 'cd_eqpt', 'nr_sequ_even', 'iproxmed').toPandas()

root
 |-- nr_idef_secu: string (nullable = true)
 |-- cd_eqpt: string (nullable = true)
 |-- nr_sequ_even: string (nullable = true)
 |-- dh_even: timestamp (nullable = true)
 |-- nr_eventos: integer (nullable = false)
 |-- mediana: double (nullable = true)
 |-- percentil_max: double (nullable = true)
 |-- iproxmed: double (nullable = true)

root
 |-- nr_idef_secu: string (nullable = true)
 |-- cd_eqpt: string (nullable = true)
 |-- nr_sequ_even: string (nullable = true)
 |-- dh_even: timestamp (nullable = true)
 |-- nr_eventos: integer (nullable = false)
 |-- percentil_max: double (nullable = true)
 |-- iproxmed: double (nullable = true)



In [275]:
# transpondo evento por tipo OK! 
# OBS: a funcao .reset_index abaixo é necessário para evitar a mesclagem das células fixas
pd_feat = pd.pivot_table(pd_pivot,index=["nr_idef_secu","cd_eqpt","dh_even"],
                         columns=["nr_sequ_even"], margins=False, values=["iproxmed"], fill_value=0).reset_index()
# acertando nomes das colunas geradas em linhas diferentes

pd_feat.columns = [c[1] if c[1] else c[0] for c in pd_feat.columns.tolist()]


# criar coluna com data como string para posterior transformação para dataframe
pd_feat = pd_feat.assign(data_even = lambda x: x['dh_even'].dt.strftime('%Y-%m-%d') ).drop('dh_even', axis=1)
pd_feat

# salva como csv pois conversao para Dataframe Spark apresenta erro. provavel falta de memoria
#pd_feat.to_csv(tabelao_out)

df_feat_alt = sqlCtx.createDataFrame(pd_feat)
# FALTA converter data em timedate de novo (converter para string no Pandas antes)

df_feat_alt.take(5)

[Row(nr_idef_secu=u'1053', cd_eqpt=u'10001', 5603=1000, 5605=0, 5614=0, 5620=0, 5622=0, data_even=u'2016-05-10'),
 Row(nr_idef_secu=u'1053', cd_eqpt=u'10001', 5603=1000, 5605=0, 5614=0, 5620=0, 5622=0, data_even=u'2016-05-16'),
 Row(nr_idef_secu=u'1053', cd_eqpt=u'10001', 5603=1000, 5605=0, 5614=0, 5620=0, 5622=0, data_even=u'2016-05-19'),
 Row(nr_idef_secu=u'1053', cd_eqpt=u'10001', 5603=1000, 5605=0, 5614=0, 5620=1000, 5622=0, data_even=u'2016-05-21'),
 Row(nr_idef_secu=u'1053', cd_eqpt=u'10001', 5603=0, 5605=0, 5614=1000, 5620=1000, 5622=0, data_even=u'2016-05-22')]

In [276]:
df_feat_alt = df_feat_alt.withColumn('data_even', df_feat_alt["data_even"].cast(TimestampType()))
df_tidy = df_feat_alt\
    .withColumn('nr_idef_secu', trim(df_feat_alt.nr_idef_secu))

#df_tidy = df_feat_alt \
#    .join(df_poc, (df_feat_alt['nr_idef_secu'] == df_poc['agencia']) &
#            (df_feat_alt['data_even'] == df_poc['data_cad']), 'left')

In [238]:
#df_tidy.orderBy(col('agencia').desc())\
#    .filter(col('agencia') == col('nr_idef_secu'))\
#    .select('agencia').distinct().toPandas()

In [239]:
# VER PORQUE LOOP ABAIXO NAO FUNCIONA
#for even in l_evenpoc:
#   df_tidy_dbl = df_tidy.withColumn(even, df_tidy[even].cast(DoubleType()))

In [277]:
df_tidy_dbl.printSchema()

root
 |-- nr_idef_secu: string (nullable = true)
 |-- cd_eqpt: string (nullable = true)
 |-- 5603: double (nullable = true)
 |-- 5605: double (nullable = true)
 |-- 5614: double (nullable = true)
 |-- 5620: double (nullable = true)
 |-- 5622: double (nullable = true)
 |-- data_even: timestamp (nullable = true)
 |-- max_evento: double (nullable = true)
 |-- label: double (nullable = false)



In [278]:
# abaixo falta testar se existe o evento antes de fazer o cast
# ou loop por indices
df_tidy_dbl = df_tidy\
    .withColumn('5603', df_tidy['5603'].cast(DoubleType()))\
    .withColumn('5605', df_tidy['5605'].cast(DoubleType()))\
    .withColumn('5614', df_tidy['5614'].cast(DoubleType()))\
    .withColumn('5620', df_tidy['5620'].cast(DoubleType()))\
    .withColumn('5622', df_tidy['5622'].cast(DoubleType()))

In [279]:
df_tidy_dbl.printSchema()

root
 |-- nr_idef_secu: string (nullable = true)
 |-- cd_eqpt: string (nullable = true)
 |-- 5603: double (nullable = true)
 |-- 5605: double (nullable = true)
 |-- 5614: double (nullable = true)
 |-- 5620: double (nullable = true)
 |-- 5622: double (nullable = true)
 |-- data_even: timestamp (nullable = true)



In [280]:
df_tidy_dbl.columns[2:-1]

['5603', '5605', '5614', '5620', '5622']

In [281]:
from functools import reduce
def superMax(*cols):
    return reduce(lambda a, b: __builtin__.max(a,b) ,cols)

add = udf(superMax)

df_tidy_dbl = df_tidy_dbl.withColumn('max_evento', add(*[df_tidy_dbl[x] for x in df_tidy_dbl.columns[2:-1]]))
   

In [282]:
df_tidy_dbl = df_tidy_dbl.withColumn('max_evento', df_tidy_dbl["max_evento"].cast(DoubleType()))

In [283]:
df_tidy_dbl = df_tidy_dbl.withColumn("label",
    when((df_tidy_dbl["max_evento"] > 1000.0), 1.0)\
        .otherwise(0.0))

In [284]:
df_tidy_dbl.groupBy("label").count().orderBy("label").show()

+-----+-----+
|label|count|
+-----+-----+
|  0.0|  683|
|  1.0|    8|
+-----+-----+



In [285]:
assemblerTrain = VectorAssembler(
    inputCols=l_evenpoc,
   outputCol="features")

In [286]:
#               ,"1617", "1689","2433", "1746", "1568","883", "2163"],
 
df_tmp = assemblerTrain.transform(df_tidy_dbl)

In [287]:
testData = df_tmp.select("label", "features")
testData.printSchema()

root
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)



In [288]:
testData.take(3)

[Row(label=0.0, features=SparseVector(5, {0: 1000.0})),
 Row(label=0.0, features=SparseVector(5, {0: 1000.0})),
 Row(label=0.0, features=SparseVector(5, {0: 1000.0}))]

### Avaliação do Modelo
Avalia o modelo usando os dados separados para teste, obtendo o erro de teste (1 - acurácia), a área sob a curva ROC (AUC) e a taxa Precision/Recall.

In [289]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
predictions = model.transform(testData)

Py4JJavaError: An error occurred while calling o1443.transform.
: java.util.NoSuchElementException: None.get
	at scala.None$.get(Option.scala:313)
	at scala.None$.get(Option.scala:311)
	at org.apache.spark.ml.feature.VectorIndexerModel$$anonfun$transformSchema$6.apply(VectorIndexer.scala:364)
	at org.apache.spark.ml.feature.VectorIndexerModel$$anonfun$transformSchema$6.apply(VectorIndexer.scala:363)
	at scala.Predef$.require(Predef.scala:233)
	at org.apache.spark.ml.feature.VectorIndexerModel.transformSchema(VectorIndexer.scala:362)
	at org.apache.spark.ml.PipelineStage.transformSchema(Pipeline.scala:62)
	at org.apache.spark.ml.feature.VectorIndexerModel.transform(VectorIndexer.scala:340)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:57)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:606)
	at py4j.refl

In [336]:
evaluator = BinaryClassificationEvaluator()

In [337]:
evaluator.explainParams()

'labelCol: label column name (default: label)\nmetricName: metric name in evaluation (areaUnderROC|areaUnderPR) (default: areaUnderROC)\nrawPredictionCol: raw prediction (a.k.a. confidence) column name (default: rawPrediction)'

In [338]:
auroc = evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})
print("AUC = %g " % (auroc))

AUC = 1 


In [339]:
precrec = evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderPR"})
print("Precision/Recall = %g " % (precrec))

Precision/Recall = 1 


In [340]:
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g " % (1.0 - accuracy))

Test Error = 0 


In [341]:
predictions\
    .select("prediction", "indexedLabel", "features", "probability")\
    .groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|       1.0|    1|
|       0.0|  215|
+----------+-----+



In [203]:
predictions.printSchema()

root
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- indexedLabel: double (nullable = true)
 |-- indexedFeatures: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = true)



In [342]:
predictions.take(10)

[Row(label=0.0, features=DenseVector([1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0]), indexedLabel=0.0, indexedFeatures=DenseVector([1000.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1000.0, 0.0, 0.0]), rawPrediction=DenseVector([20.0, 0.0]), probability=DenseVector([1.0, 0.0]), prediction=0.0),
 Row(label=0.0, features=DenseVector([1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0]), indexedLabel=0.0, indexedFeatures=DenseVector([1000.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1000.0, 0.0, 0.0]), rawPrediction=DenseVector([20.0, 0.0]), probability=DenseVector([1.0, 0.0]), prediction=0.0),
 Row(label=0.0, features=DenseVector([1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0]), indexedLabel=0.0, indexedFeatures=DenseVector([1000.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1000.0, 0.0, 0.0]), rawPrediction=DenseVector([20.0, 0.0]), probability=DenseVector([1.0, 0.0]), prediction=0.0),
 Row(label=0.0, features=DenseVector([1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000

Abaixo são mostradas as primeiras linhas da tabela de previsão gerada pelo modelo

In [343]:
pd_pred = predictions.select("prediction", "indexedLabel", "features", "probability").toPandas()

In [186]:
# 6. dividir dados em treino e teste
# selecionando sample estratificada
#df_tidy_dbl.groupBy("label").count().orderBy("label").show()
# N: 0.9995   S: 0.000499
# Balanceamento com marcação anterior de report de POC
# N: 0.0110104 S: 0.98898
# Balanceamento com marcação por evento fora do percentil de POC
# N:  S: 

In [187]:
#df_tidy_dbl.groupBy(["nr_idef_secu", "label"]).count().orderBy("nr_idef_secu").show()

In [346]:
# salving the resulting dataframe
#type(poc_predicted)
pd_pred.to_csv(previsao)
#df.write.csv('mycsv.csv')

In [208]:
# estratégia
# 1. obter a média diária da agência + atm (trocar para mediana) para cada evento
# 2. do log, somar o número de ocorrências por agencia + atm + evento agrupado por dia
# 3. montar tabelao inicial e criar coluna count/media para ver índice de desvio da média (FEATURE) por agencia/atm
# 4. criar coluna LABEL concatenando com arquivo POC por agencia + data cad (atm nao confiavel no arquivo)
# obs: desvio > 1 indica percentual acima da média e vice-versa
# 5. fazer o pivot como acima

# trocar media por mediana (FALTA)
# usar mediana para gerar o tabelao (ou percentil 75% ou soma contante) (FALTA)
# dividir pelo count para ter a feature percentil/count (FALTA)
# transposicao (filtrar tabelao por agencia + atm e join dois a dois (em loop?))
# filtrar os 3 dias após a data de poc para tirar efeitos de setup   
# 

In [191]:
# matriz de confusão
#poc_predicted

## confusion matrix

In [205]:
def confusion_matrix(prevCol, labelCol):
    if prevCol == 1.0 and labelCol == 1.0:
        return 'TP'
    else:
        if prevCol == 1.0 and labelCol == 0.0:
            return 'FP'
        else:
            if prevCol == 0.0 and labelCol == 0.0:
                return 'TN'
            else:
                if prevCol == 0.0 and labelCol == 1.0:
                    return 'FN'
            
createNewColFromTwo = udf(confusion_matrix, DoubleType())
df_conf_matr = predictions.withColumn('conf_matr', createNewColFromTwo(predictions['prediction'],
            predictions['indexedLabel']))

In [386]:
#df_conf_matr.show()

In [387]:
# matriz de confuao
#df_conf_matr.groupBy("conf_matr").count().show()

In [388]:
# Sensibility - habilidade do modelo classificar corretamente um POC
# Probability of being test positive when POC present.
# TP / (TP + FN)
#sensitivity = float(df_rates.collect()[0][1]) / (df_rates.collect()[0][1] + df_rates.collect()[2][1])
# Specificity - habilidade do modelo classificar corretamente a ausência de POC 
#Probability of being test negative when POC absent.
# TN / (TN + FP)
#specificity = float(df_rates.collect()[1][1]) / (df_rates.collect()[1][1] + df_rates.collect()[3][1])
# precisao
# TP / (TP + FP)
#precisao = float(df_rates.collect()[0][1]) / (df_rates.collect()[0][1] + df_rates.collect()[3][1])

#print ("\nSentitividade ou Recall: " + str("%.2f" % (sensitivity)))
#print ("\nEspecificidade         : " + str("%.2f" % (specificity)))
#print ("\nPrecisão               : " + str("%.2f" % (precisao)))

In [389]:
#df_rates.toPandas()

In [ ]:
# TESTES
predictions.filter((col("prediction") == 0.0) & (col("label") == 1.0)).count()

In [ ]:
df_rates.collect()

In [ ]:
predictions.filter((col("prediction") == 1.0) & (col("label") == 1.0)).count()

In [144]:
predictions.filter((col("prediction") == 1.0) & (col("label") == 0.0)).count()

621

In [107]:
predictions.filter((col("prediction") == 0.0) & (col("label") == 0.0)).count()

19698

In [390]:
# checagem do marcador de POC contra POCs reportados
# somente POC retencao de cartão, D-90
# trazer número de POCs para a chave agencia
# encotradas 52 agências com POC
#df_p = df_poc.groupBy('agencia').count().orderBy('agencia')
#df_p.toPandas()
# agora comparar com o arquivo gerado com label

In [391]:
# foram encontradas agencias de POC nos eventos abaixo filtrados
#df_l = df_feat.where(col('label') == 1.0).groupBy('nr_idef_secu').count().orderBy('nr_idef_secu')

#df_l.toPandas()

In [392]:
# checar se selecionou realmente somente agencias com POC do log geral
#df_evenpoc.groupBy('nr_idef_secu').count().orderBy('nr_idef_secu').toPandas()

In [393]:
# checar quantas agencias reportadas como POC cairam no log com nova marcação (deveriam aparecer todos )
#df_j = df_l \
#    .join(df_p, (df_l['nr_idef_secu'] == df_p['agencia']), 'inner')
    
    

#df_j.limit(10).toPandas()